In [12]:
course_list= ["a"]
course_list.remove("a")
print(course_list)
print(len(course_list))

[]
0


In [34]:
(course, group) = "aa-bb-cc".split("-")
print(course)
print(group)

aa
bb


In [52]:
hierarchy = "aa>bb>cc".split(">")
for var in hierarchy:
    print(var)
    if(var=="bb"):
        hierarchy=["dd"]
    print(hierarchy)

aa
['aa', 'bb', 'cc']
bb
['dd']
cc
['dd']


In [5]:
a="aaa"
print(a.split(">"))

['aaa']
